In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Extraction with Generative Models on Vertex AI 

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/text_extraction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/text_extraction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/language/examples/prompt-design/text_extraction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

Text extraction is a process of extracting text from a document. This can be done manually or automatically. Manual text extraction is the process of reading the document and copying the text into a new document. Automatic text extraction is the process of using software to extract the text from the document.

Text extraction can be used for a variety of purposes. One common purpose is to convert documents into a machine-readable format. This can be useful for storing documents in a database or for processing documents with software. Another common purpose is to extract information from documents. This can be useful for finding specific information in a document or for summarizing the content of a document.

Large language models (LLMs) are good for text extraction because they are trained on massive datasets of text and code, which allows them to learn the relationships between words and phrases. They can also understand the context of text and generate text, which allows them to extract information that is not explicitly stated or fill in the gaps in text that is missing information. The answers from LLMs can also be further improved through methods like few-shot prompting.

Learn more about extraction prompts in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/extraction-prompts).

### Objective

In this tutorial, you will learn how to use generative models to extract the information from text by working through the following examples:
- Google Pixel technical specifications extraction
- WiFi troubleshooting with constraints
- Respond to inquiries in character
- Converting an ingredients list to JSON format
- Organizing the results of a text extraction


### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI Generative AI Studio

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Getting Started

### Install Vertex AI SDK

In [ ]:
!pip install google-cloud-aiplatform google-cloud-translate --upgrade --user

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top. 

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Import libraries


**Colab only:** Uncomment the following cell to initialize the Vertex AI SDK. For Vertex AI Workbench, you don't need to run this.  

In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
from vertexai.preview.language_models import TextGenerationModel

### Import models

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

### Create the translation wrapper function

In [ ]:
from google.cloud import translate

project_id = !gcloud config list project
project_id = project_id[1].split('=')[1].strip()
parent = f'projects/' + project_id


def traduza(texto, idioma_destino):
    client = translate.TranslationServiceClient()

    response = client.translate_text(
        parent=parent,
        contents=[texto],
        target_language_code=idioma_destino,
        mime_type="text/plain"
    )

    return response.translations[0].translated_text

## Text Extraction

### Google Pixel technical specifications extraction

In this example, you try to extract the technical specifications of a Pixel phone from text in JSON format using the PaLM API.

In [ ]:
prompt = traduza("""
Extraia as especificações técnicas do texto abaixo no formato JSON.

Texto: Google Nest WiFi, velocidade de rede de até 1200Mpbs, frequências de 2,4 GHz e 5 GHz, protocolo WP3
JSON: {
   "product":"Google Nest WiFi",
   "velocidade":"1200Mpbs",
   "frequências": ["2,4GHz", "5GHz"],
   "protocolo":"WP3"
}

Texto: Google Pixel 7, rede 5G, 8 GB de RAM, processador Tensor G2, 128 GB de armazenamento, Lemongrass
JSON:
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text, "pt")
)

### WiFi troubleshooting with constraints

In this example, you ask the generative model to answer a question about troubleshooting a Google WiFi router based on the description of the different status lights on the router. The model will only be able to respond with the text that was provided, which helps to prevent it from generating potentially harmful or incorrect answers. Here is how you can do this using the PaLM API.

In [ ]:
prompt = traduza("""
Responda a questão usando o texto abaixo. Responda apenas com o texto fornecido.
Pergunta: O que devo fazer para consertar meu WiFi desconectado? A luz no meu roteador Google WiFi está amarela e piscando lentamente.

Texto:
Cor: Sem luz
O que significa: o roteador não tem energia ou a luz foi reduzida no aplicativo.
O que fazer:
Verifique se o cabo de alimentação está conectado corretamente ao roteador e a uma tomada que esteja funcionando.
Se o seu dispositivo já estiver configurado e a luz estiver apagada, verifique as configurações de brilho da luz no aplicativo.
Se ainda não houver luz, entre em contato com o suporte ao cliente Google WiFi.

Cor: branco sólido, sem luz, branco sólido
O que significa: O dispositivo está inicializando.
O que fazer:
Aguarde o dispositivo inicializar. Isso leva cerca de um minuto. Quando terminar, ele pulsará lentamente em branco, informando que está pronto para a configuração.

Cor: Branco de pulsação lenta
O que significa: O dispositivo está pronto para configuração.
O que fazer:
Use o app Google Home para configurar seu roteador.

Cor: Branco sólido
O que significa: o roteador está online e está tudo bem.
O que fazer:
Você está online. Aproveitar!

Cor: amarelo pulsando lentamente
O que significa: há um erro de rede.
O que fazer:
Verifique se o cabo Ethernet está conectado ao roteador e ao modem e se ambos os dispositivos estão ligados. Pode ser necessário desconectar e conectar cada dispositivo novamente.

Cor: Amarelo piscando rápido
O que significa: você está pressionando o botão de redefinição e redefinindo a configuração original deste dispositivo.
O que fazer:
Se você continuar pressionando o botão de reinicialização, após cerca de 12 segundos, a luz ficará amarela sólida. Quando estiver amarelo sólido, solte o botão de redefinição de fábrica.

Cor: amarelo sólido
O que significa: o roteador está sendo redefinido para as configurações de fábrica.
O que fazer:
Isso pode levar até 10 minutos. Quando terminar, o dispositivo será reinicializado e começará a piscar em branco, informando que está pronto para a configuração.
Imagem Luz vermelha constante Vermelho sólido Algo está errado. Falha crítica. Reinicie o roteador de fábrica. Se a luz permanecer vermelha, entre em contato com o suporte ao cliente WiFi.
""", "en")
print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=1, top_p=0.8).text, "pt")
)

### Respond to inquiries in character

Now, you instruct the generative model to pretend to be Klara, a person. You will also tell the model about Klara's personality traits. Then, you will ask the model to answer a question as Klara would answer it.

In [ ]:
prompt = traduza("""
Você é a Klara.
Klara é gestora de investimentos.
Klara só responde se Klara tiver certeza de que está correto.
Klara responde à pergunta do usuário com base nos resumos das páginas abaixo.
Klara gera a ID de referência em que Klara encontrou a resposta para cada frase da resposta.
Klara também resume a parte onde se encontram as informações.

Resumos das páginas: ['ID de referência 1. O UBS quer ser visto como um banco global com raízes suíças, não apenas um banco europeu. O novo presidente do conselho, \
                    Colm Kelleher, e o CEO, Ralph Hamers, realizaram uma série de reuniões com influentes gestores de fundos dos EUA para aumentar suas participações \
                    no banco. O UBS é um dos bancos europeus mais valiosos com uma relação preço/valor contábil de 1.', 'ID de referência 2. Credit Suisse vende \
                    participação de 30% na gestora de ativos suíça Energy Infrastructure Partners. A EIP é especializada em investimentos de capital de longo prazo \
                    para ativos de infraestrutura de energia e renováveis em larga escala. A transação pode fazer parte de um esforço contínuo do banco suíço para \
                    reduzir ainda mais a lacuna de capital.' O príncipe Mohammed injetaria cerca de US$ 500 milhões na cisão do CS First Boston. Outros investidores \
                    podem incluir o ex-chefe do Barclays, Bob Diamonds, Atlas Merchant Capital. O Saudi National Bank já possui uma participação de 9,9 por cento na \
                    problemática instituição suíça. . O Wall Street Journal informou que a Apollo também está interessada em investir no CS First Boston. A divisão \
                    de banco de investimento de banco de investimento está prestes a ser desmembrada em uma nova unidade .', 'ID de referência 5. O banco privado \
                    Pictet, com sede em Genebra, contratou uma nova equipe em seu negócio de investimento no mercado privado de bilhões de dólares . Edmund Buckley, \
                    Nikolaus Hubmann, Sean Howard, Jan Dreesen e Hugo Hickson trabalharão na área de investimentos diretos em private equity no futuro. O \
                    comprometimento da equipe de Buckley pode ser visto como um golpe para Elif Aktuğ, parceiro da Pictet .', 'ID de referência 6. O UBS quer ser visto \
                    como um banco global com raízes suíças, não apenas um banco europeu . O novo presidente do conselho, Colm Kelleher, e o CEO, Ralph Hamers, \
                    realizaram uma série de reuniões com influentes gestores de fundos dos EUA para aumentar suas participações no banco. O UBS é um dos bancos \
                    europeus mais valiosos com uma relação preço/valor contábil de 1 .', 'ID de referência 7. A Apollo Global Management, sediada nos EUA, está \
                    entre um grupo de empresas financeiras em negociações com o Credit Suisse sobre uma participação no banco de investimento renovado. O Wall \
                    Street Journal informou que a Apollo também está interessada em investir no CS First Boston. A divisão de banco de investimento de bancos de \
                    investimento está definida para ser dividida em uma nova unidade.', 'ID de referência 8. A subsidiária da Apex com sede em Luxemburgo European \
                    Depositary Bank (EDB) e o especialista em securitização Gentwo firmam parceria . A EDB e a Gentwo oferecerão agentes pagadores e serviços \
                    bancários a investidores terceirizados em todo o mundo. Gentwo desenvolve plataformas para gerentes de ativos, bancos, escritórios familiares e \
                    capitalistas de risco .', 'Referência ID 9. O SNB tem perseguido planos para adquirir participações em instituições financeiras europeias e \
                    americanas há algum tempo . Credit Suisse, Julius Baer, Standard Chartered e o Asian DBS Group também foram mencionados como possíveis alvos .', \
                    'Referência ID 10. Zuercher Kantonalbank pretende expandir seus negócios de private banking, inclusive no exterior. O CEO Urs Baumann vê o Credit \
                    Suisse como um parceiro confiável para o banco estatal. A Baumann continuará a se concentrar em reduzir a dependência dos bancos no negócio de \
                    diferencial de juros em uma tentativa de diversificar seus negócios.']

Pergunta do usuário: Os bancos globais estão investindo em iniciativas ESG?
Resposta da Klara:
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=40, top_p=0.8).text, "pt")
)

###  Converting an ingredients list to JSON format

Suppose that you want to itemize ingredients in recipes to enter into a database, which requires a well-formatted output like JSON. This can be done using a generative model in the following way:

In [ ]:
prompt = traduza("""
Extraia os ingredientes da seguinte receita. Retorne os ingredientes no formato JSON com as chaves: ingrediente, quantidade, tipo.

Ingredientes:
* 1 colher de sopa de azeite
* 1 cebola picada
* 2 cenouras picadas
* 2 talos de aipo picados
* 1 colher de chá de cominho moído
* 1/2 colher de chá de coentro moído
* 1/4 colher de chá de açafrão em pó
* 1/4 colher de chá de pimenta caiena (opcional)
* Sal e pimenta a gosto
* 425g de feijão preto, lavado e escorrido
* 425g de feijão, lavado e escorrido
* 420g lata de tomate em cubos, não escorrido
* 280g lata de tomate em cubos com pimentões verdes, não drenados
* 4 xícaras de caldo de legumes
* 1 xícara de coentro fresco picado
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text, "pt")
)

### Organizing the results of a text extraction

In this section, you extract the information you want from a block of text and organize it in a structured way, such as separating it by commas. Here you use few-shot prompting to guide the model to format your outputs to be separated by commas.


In [ ]:
prompt = traduza("""
Mensagem: Rachel Green (Jennifer Aniston), uma mulher protegida, mas amigável, foge do dia do casamento e de uma vida rica, mas insatisfatória, e encontra a amiga de infância Monica Geller (Courteney Cox), uma chef tensa, mas atenciosa.
Rachel se torna garçonete no café Central Perk do West Village depois que ela se muda para o apartamento de Monica acima do Central Perk e se junta ao grupo de amigas solteiras de Monica na casa dos 20 anos:
a colega de quarto anterior Phoebe Buffay (Lisa Kudrow), uma estranha massagista e musicista; o vizinho Joey Tribbiani (Matt LeBlanc), um ator obstinado, mas leal e esforçado; O colega de quarto de Joey, Chandler Bing ( Matthew Perry ),
um processador de dados sarcástico e autodepreciativo; e o irmão mais velho de Monica e colega de quarto de faculdade de Chandler, Ross Geller (David Schwimmer), um paleontólogo doce, mas inseguro.

Extraia os personagens e os atores que os interpretaram da mensagem acima:
Rachel Green - Jennifer Aniston, Monica Geller - Courteney Cox, Phoebe Buffay - Lisa Kudrow, Joey Tribbiani - Matt LeBlanc, Chandler Bing - Matthew Perry, Ross Geller - David Schwimmer

Mensagem: Jogos como xadrez, pôquer, Go e muitos videogames sempre foram um terreno fértil para a pesquisa de IA. Diplomacia é um jogo de negociação e formação de alianças para sete jogadores, jogado em um antigo mapa da Europa dividido
em províncias, onde cada jogador controla várias unidades (regras de Diplomacia). Na versão padrão do jogo, chamada Press Diplomacy, cada turno inclui uma fase de negociação, após a qual todos os jogadores revelam seu
movimentos escolhidos simultaneamente. O coração da Diplomacia é a fase de negociação, onde os jogadores tentam chegar a um acordo sobre seus próximos movimentos. Por exemplo, uma unidade pode apoiar outra unidade, permitindo-lhe superar a resistência de outras unidades,
conforme ilustrado aqui: Abordagens computacionais para Diplomacia têm sido pesquisadas desde a década de 1980, muitas das quais foram exploradas em uma versão mais simples do jogo chamado No-Press Diplomacy, onde a comunicação estratégica entre
jogadores não é permitido. Os pesquisadores também propuseram protocolos de negociação amigáveis ao computador, às vezes chamados \342\200\234Restricted-Press\342\200\235.

Extraia a definição de Diplomacia:
Um jogo de negociação e formação de alianças para sete jogadores


Mensagem: Em 2016, quando não estávamos usando simulação e estávamos usando uma pequena configuração de laboratório de robôs industriais para aprender a pegar pequenos objetos como brinquedos, chaves e utensílios domésticos do dia a dia, levou o equivalente a
quatro meses para um robô aprender a realizar uma pegada simples com uma taxa de sucesso de 75%. Hoje, um único robô aprende a realizar uma tarefa complexa como abrir portas com uma taxa de sucesso de 90%% em menos de um dia
de aprendizado do mundo real. Ainda mais empolgante, mostramos que podemos desenvolver os algoritmos e aprendizados da abertura de portas e aplicá-los a uma nova tarefa: endireitar cadeiras em nossos cafés. Este progresso nos dá
esperamos que nossa tentativa de construir robôs de aprendizagem de uso geral seja possível.

Extraia as taxas de sucesso dos robôs em 2016 e hoje, respectivamente:
75%, 90%

Mensagem: A CapitalG foi fundada há uma década para capacitar empreendedores com a experiência incomparável da Alphabet e do Google em crescimento.
Temos o privilégio de compartilhar as lições aprendidas ao ajudar a escalar Google, Stripe, Airbnb, CrowdStrike, Databricks e Zscaler com a próxima onda de empresas de tecnologia geracional, talvez incluindo a sua.
A Alphabet é nosso único LP e fornece capital paciente e de longo prazo. Como um fundo de crescimento independente, nossas prioridades estão alinhadas com nossos empreendedores. As empresas CapitalG atingiram o ajuste do mercado de produtos e estão prontas para escalar. Mantemos um portfólio pequeno e concentrado para que cada empresa receba capital substancial e suporte prático.

Extraia as empresas financiadas pela CapitalG:
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=1, top_p=0.8).text, "pt")
)

As you can see in the output above, based on the few-shot prompt, you should see the names of companies funded by CapitalG.